In [1]:
import psycopg2
#from connection_string_p12 import connection_string1
from collections import defaultdict
from datetime import datetime
import re
from collections import Counter
from estnltk import Text
import pickle
from tqdm import tqdm
import sys
import os

In [2]:
sys.path.append(re.sub('cda_data_cleaning/fact_extraction/diag_text_parsing/CD_diag_explorations', '', os.getcwd()))

In [3]:
#from cda_data_cleaning.fact_extraction.diag_text_parsing.development.taggers.diagnosis_text_tagger import DiagnosisTextTagger
from cda_data_cleaning.fact_extraction.diag_text_parsing.development.taggers.cancer_stage_tagger import CancerStageTagger
#from cda_data_cleaning.fact_extraction.measurement_extraction.taggers.robust_date_number_tagger.robust_date_number_tagger import RobustDateNumberTagger

INFO:base_parser.py:30: Loaded []


In [4]:
pwd

'/home/dage/Desktop/cda-data-cleaning/cda_data_cleaning/fact_extraction/diag_text_parsing/CD_diag_explorations'

In [4]:
real_extras = pickle.load(open("../all_diag_explorations2/real_extras2.pickle", "rb"))

In [5]:
cd_texts = Counter()
for thing, count in real_extras.items():
    if thing[1] == 'C00-D48': 
        cd_texts[thing] = count

In [7]:
cd_texts.most_common()

[(('Pk', 'C00-D48'), 110),
 (('Kõigi paikmete hemangioom', 'C00-D48'), 81),
 (('E pk', 'C00-D48'), 60),
 (('Umbsool', 'C00-D48'), 46),
 (('A pk', 'C00-D48'), 43),
 (('Neer', 'C00-D48'), 36),
 (('I pk', 'C00-D48'), 34),
 (('Nahk', 'C00-D48'), 31),
 (('Neerupealis', 'C00-D48'), 27),
 (('E pahaloomuline kasvaja', 'C00-D48'), 26),
 (('Pahaloomuline kasvaja', 'C00-D48'), 22),
 (('Rinna pahaloomuline kasvaja', 'C00-D48'), 15),
 (('IV st.', 'C00-D48'), 13),
 (('Lümfoplasmotsütaarne lümfoom', 'C00-D48'), 12),
 (('Kõri', 'C00-D48'), 11),
 (('E kolangio- e sapisoonekartsinoom', 'C00-D48'), 11),
 (('Th 6-11 paravertebraalne perifeerne primitiivne neuroektodermaalne tuumor/ekstraossaalne Ewingi sarkoom ; c T2 N0 M1B IVBst\nTh 6-11 paravertebraalne perifeerne primitiivne neuroektodermaalne tuumor/ekstraossaalne Ewingi sarkoom',
   'C00-D48'),
  10),
 (('IgG I A st.', 'C00-D48'), 10),
 (('Adenocarcinoma  prostatae GS 3+4=7', 'C00-D48'), 10),
 (('Pärasool', 'C00-D48'), 10),
 (('E pk -26.08.2013- aden

In [8]:
stages = []
st = []
for i in cd_texts:
    stage_old = re.search('(?P<stage>(I[IV]+)|(I[^A-ZÜÕÖÄŠŽa-züõöäš])) *(?P<letter>[abcABC])? *(?P<st>st)?', i[0])
    stage = re.search('(I[IV]+ *[ABCabc]) *(st)?', i[0])
    stage2 = re.search('(I[IV]+) *st', i[0])
    stage3 = re.search('([0-5] *[ABCabc]) *(st)?', i[0])
    stage4 = re.search('([0-5]) *st', i[0])
    stage5 = re.search('st\.? *([0-5] *[ABCabc]*)', i[0])
    stage6 = re.search('st\.? *(I[IV]+ *[ABCabc]*)', i[0])
    #print(stage)
    #print(i)
    if stage:
        stages.append((i[0], stage.group('stage'), stage.group('letter'), stage.group('st')))
    #if re.search('(^|\s|[-,])(?=[MDCLXVI])M*(C[MD]|D?C*)(X[CL]|L?X*)(I[XV]|V?I*)($|\s|[-,])', i[0]):
        #roman.append(i)
    #elif 'st' in i[0]:
    #    st.append(i)

IndexError: no such group

In [ ]:
rooma täht st?
rooma st
araabia täht st?
araabia st
st rooma täht?
st araabia täht?

In [ ]:
# 0 staadium
# araabia numbritega staadiumid
# st või st. enne numbrit
# tähega staadiumid ilma st-ta
# I, millele eelnevad valed asjad (suured tähed/nr'd)
# vv
# tee regex_tagger
# jooksuta diagnoositekstide lisadel
# jooksuta vähidiagnoosiga tekstiväljadel

In [37]:
len(stages)

139

In [39]:
for i in stages:
    if i[3] is None:
        print(i)

('4ac1c1PER IV', 'IV', None, None)
('Carcinima ovarii pT cNOMO st. IIIC.', 'III', 'C', None)
('Carcinoma anaplasticum ventriculi pT3N2M0 G3  st.IIb', 'II', 'b', None)
('Kolangio- e spisoonekartsinoom 2c0c1 IV   2', 'IV', None, None)
('B1 thymoma; Masaoka III', 'III', None, None)
('Adenokartsinoom; cT3cN1M0 IIIB enne neoadjuvantset kemoradiatsiooni', 'III', 'B', None)
('Difuusne B-suurrakk-lümfoom, IIEB', 'II', None, None)
('Juustega peanaha ja kaelanaha pahaloomuline melanoom pT3b Nx M0, R0 - st IIB', 'II', 'B', None)
('MGB G IV paremas otsmiku- ja oimusagaras', 'IV', None, None)
('. st IIIC', 'III', 'C', None)
('Myomae uteri submucosum I - 0 st (1,5x2,5cm); II - 1 st (D 2,5-3cm).', 'I ', None, None)
('Se pahaloomuline kasvaja; III vatsakese ja thalamuse piirkonna maligniseerunud epidermoidt', 'III', None, None)
('I pk', 'I ', None, None)
('Vasaku rinna kartsinoom ÖRpos, PRpos, HER2 neg; p T1C N1MI M0 G1 R0 <ANONYM id="1" type="per" morph="_Y_ sg el"/>\nVasaku rinna kartsinoom ÖRpos, P

In [9]:
len(st)

204

In [10]:
st

[('(transformatsiooniga üle müelodüsplastilise sündroomi (RAEB2))', 'C00-D48'),
 ('PK (stenditud)', 'C00-D48'),
 ('- Ca. sigillocellulare ventriculi cum metastasibus.', 'C00-D48'),
 ('Mitte-hodgkini tüüpi lümfoomi muud täpsustatud tüübid', 'C00-D48'),
 ('.Bronchocarcinoma lobi inf. pulmonis dex. pT2 pNo MO st 1b ( operata 27.10.2009a ). Intoxicatio.',
  'C00-D48'),
 ('Aju metastaasidega', 'C00-D48'),
 ('(IVB st)', 'C00-D48'),
 ('Täpsustamata maks', 'C00-D48'),
 ('Kaksteistsõrmik', 'C00-D48'),
 ('IVBst. ALK+', 'C00-D48'),
 ('----metastaasidega', 'C00-D48'),
 ('Carcinima ovarii pT cNOMO st. IIIC.', 'C00-D48'),
 ('Täpsustamata sapiteed', 'C00-D48'),
 ('Vasema munasarja tsüst', 'C00-D48'),
 ('Carcinoma anaplasticum ventriculi pT3N2M0 G3  st.IIb', 'C00-D48'),
 ('Vasaku abaluu piirkonnas; p T1B N2C M0 R0 IIIBst\nvasaku abaluu piirkonnas',
  'C00-D48'),
 ('G3a; IIIAst\nG3a', 'C00-D48'),
 ('E pahaloomuline kasvaja; mutsinoosadenokartsinoom; Staadium arvel 4B; c T4B N2 M1B IVBst\nmutsinoosadeno

In [8]:
tagger = CancerStageTagger()

In [9]:
tagged_texts = []
not_tagged_texts = []
for t in cd_texts:
    text = Text(t[0])
    tagger.tag(text)
    if len(text.stages) > 0:
        tagged_texts.append(text)
    else:
        not_tagged_texts.append(t)

In [10]:
len(tagged_texts)

127

In [11]:
len(not_tagged_texts)

787

In [12]:
tagged_texts[0].stages

Layer(name='stages', attributes=('grammar_symbol', 'regex_type', 'value', '_priority_'), spans=SL[Span('IV a st', [{'grammar_symbol': 'STAGE', 'regex_type': 'r1', 'value': 'IV a', '_priority_': 0}])])

In [13]:
for t in tagged_texts:
    print(t.text)
    print(t.stages.value)    

Adenokartsinoom: c T3 N1 M1a IV a st.
[['IV a']]
.Bronchocarcinoma lobi inf. pulmonis dex. pT2 pNo MO st 1b ( operata 27.10.2009a ). Intoxicatio.
[['1b']]
Pahaloomuline kasvaja; p T4 N0, M0 III st.
[['III']]
(IVB st)
[['IVB']]
IV st
[['IV']]
IVBst. ALK+
[['IVB']]
Carcinima ovarii pT cNOMO st. IIIC.
[['IIIC']]
Carcinoma anaplasticum ventriculi pT3N2M0 G3  st.IIb
[['IIb']]
4 c 1 c X III st.
[['III']]
Vasaku abaluu piirkonnas; p T1B N2C M0 R0 IIIBst
vasaku abaluu piirkonnas
[['IIIB']]
IgG III A st
IgG III A st
[['III A'], ['III A']]
G3a; IIIAst
G3a
[['IIIA']]
E pahaloomuline kasvaja; mutsinoosadenokartsinoom; Staadium arvel 4B; c T4B N2 M1B IVBst
mutsinoosadenokartsinoom
[['IVB']]
Th 6-11 paravertebraalne perifeerne primitiivne neuroektodermaalne tuumor/ekstraossaalne Ewingi sarkoom ; c T2 N0 M1B IVBst
Th 6-11 paravertebraalne perifeerne primitiivne neuroektodermaalne tuumor/ekstraossaalne Ewingi sarkoom
[['IVB']]
IVCst
[['IVC']]
Alajäseme, k.a puusa pahaloomuline melanoom pTx pN1b M0 - I

In [13]:
tagger.tag(Text('Myomae uteri submucosum I - 0 st (1,5x2,5cm); II - 1 st (D 2,5-3cm).')).stages[0]

text,grammar_symbol,regex_type,value,_priority_
I,STAGE_OLD,stage_1,I,1
